<a href="https://colab.research.google.com/github/Yangel300/Sistemas_Embebidos_2025-2/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os

In [ ]:
len_obj=900

def cut_0(l,len_obj):
  return l[-len_obj:]

In [ ]:
import pandas as pd

Names=["Andres","Cesar","David","Erika", "Oscar"]

path="/content"

CSV=[]
for i in Names:
  r=[]
  for j in os.listdir(f"{path}/{i}"):
    k=pd.read_csv(f"{path}/{i}/{j}")
    k=cut_0(k,len_obj)
    r.append(k)
  CSV.append(r)
#print(CSV)
#

In [ ]:
print(CSV[4][4])

     pitch (°)  roll (°)  accel_total (g)
72       3.859     1.383            0.865
73       3.880     1.379            0.865
74       3.900     1.380            0.863
75       3.923     1.382            0.859
76       3.949     1.381            0.858
..         ...       ...              ...
967     36.385   -43.993            0.971
968     36.210   -43.850            0.987
969     35.967   -43.765            1.002
970     35.663   -43.706            1.018
971     35.272   -43.687            1.032

[900 rows x 3 columns]


In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor



In [ ]:
clf=DecisionTreeClassifier(max_depth=4,random_state=2)

In [ ]:
all_data = []
all_labels = []

for person_index, person_data in enumerate(CSV):
  # Skip the first person ('Andres') for the training set
  if person_index > 0:
    for activity_index, activity_data in enumerate(person_data):
      all_data.append(activity_data)
      # Repeat the label for each sample in the dataframe
      labels_for_activity = np.full(len(activity_data), activity_index)
      all_labels.append(labels_for_activity)

# Concatenate all dataframes into a single dataframe
X_train = pd.concat(all_data, ignore_index=True)
# Concatenate all label arrays into a single 1D numpy array
y_train = np.concatenate(all_labels)


# Now prepare the test set for 'Andres'
test_data = []
test_labels = []

for activity_index, activity_data in enumerate(CSV[0]): # CSV[0] is 'Andres' data
    test_data.append(activity_data)
    labels_for_activity = np.full(len(activity_data), activity_index)
    test_labels.append(labels_for_activity)

X_test = pd.concat(test_data, ignore_index=True)
y_test = np.concatenate(test_labels) # Also flatten y_test for consistency

In [ ]:

print(X_test.shape)
print(y_test.shape)

(4500, 3)
(4500,)


In [ ]:
# The splitting of data is now handled in the previous cell (b020c243).
# We can remove the splitting logic from this cell.

print(type(X_train))
print(type(y_train))
print(X_train.shape)
print(y_train.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(18000, 3)
(18000,)


In [ ]:
print(X_train)

       pitch (°)  roll (°)  accel_total (g)
0         16.030    40.556            1.118
1         16.092    41.176            1.141
2         16.156    41.792            1.144
3         16.229    42.409            1.138
4         16.314    43.027            1.131
...          ...       ...              ...
17995     36.385   -43.993            0.971
17996     36.210   -43.850            0.987
17997     35.967   -43.765            1.002
17998     35.663   -43.706            1.018
17999     35.272   -43.687            1.032

[18000 rows x 3 columns]


In [ ]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=4, random_state=2)

In [ ]:
y_pred_clf = clf.predict(X_test)

In [ ]:
print(y_test)

[0 0 0 ... 4 4 4]


In [ ]:
print(y_pred_clf)

[2 2 2 ... 4 4 4]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_clf)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7397777777777778


In [ ]:
import pickle

# Save the trained model to a pickle file
filename = 'Cesar.pkl'
pickle.dump(clf, open(filename, 'wb'))

print(f"Model saved to {filename}")

Model saved to Cesar.pkl


In [ ]:
import pickle
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
with open('ArbolAndres.pkl', 'rb') as file:
    Andres_model = pickle.load(file)
#with open('ArbolCesar.pkl', 'rb') as file:
#    Cesar_model = pickle.load(file)
with open('ArbolDavid.pkl', 'rb') as file:
    David_model = pickle.load(file)
#with open('ArbolErika.pkl', 'rb') as file:
#    Erika_model = pickle.load(file)
with open('ArbolOscar.pkl', 'rb') as file:
    Oscar_model = pickle.load(file)

#a partir de los arboles cargados crear un random forest

In [ ]:
import numpy as np
from collections import Counter

# Assuming Andres_model, David_model, and Oscar_model were loaded successfully
# and X_test contains your test data (ensure the cell defining X_test is executed before this one)

# Make predictions with each loaded model
andres_predictions = Andres_model.predict(X_test)
david_predictions = David_model.predict(X_test)
oscar_predictions = Oscar_model.predict(X_test)
#erika_predictions = Erika_model.predict(X_test)
#cesar_predictions = Cesar_model.predict(X_test)

# Stack the predictions for majority voting
# Ensure all prediction arrays have the same shape
if not (andres_predictions.shape == david_predictions.shape == oscar_predictions.shape):
    print("Warning: Prediction shapes do not match. Majority voting might not work as expected.")
    # You might need to add padding or handle this inconsistency based on your data structure

all_predictions = np.vstack((andres_predictions, david_predictions, oscar_predictions))

# Perform majority voting for each sample
def majority_vote(predictions_for_sample):
    most_common = Counter(predictions_for_sample).most_common(1)
    return most_common[0][0]

# Apply majority voting across all samples
ensemble_predictions = np.apply_along_axis(majority_vote, axis=0, arr=all_predictions)

print("Ensemble predictions generated using majority voting from loaded models.")

# You can now evaluate ensemble_predictions against y_test
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {accuracy}")